# Deep Learning Project
Leveraging language models for text based stegonography

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch
!pip install transformers[torch]
!pip install accelerate -U
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import tensorflow as tf
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from torch.utils.data import DataLoader, Dataset
# import emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

In [ ]:
fileName = '/content/drive/MyDrive/deep/Urdu Tweets Dataset.xlsx'
# dataset = pd.read_table(fileName)
df = pd.read_excel(fileName)
df.head(10)

,Id,Text,Emotions,Category
0,1313165582020349952,.Assalam Alikum 🦋😊 اے ایمان والو میرے دشم...,"['SMILING FACE WITH SMILING EYES , 0.644']",Joy
1,1314653586509488128,- '' 🍀 🌾 🌴 '' - ﺍﮬﺪِﻧَﺎﻟﺼِّﺮَﺍﻁَ ﺍَﻟﻤُﺴﺘَﻘِﯿﻢ...,"['HEAVY BLACK HEART , 0.746', 'SPARKLES , 0.35...",NaN
2,1311404383071067904,💕 میں نہیں کرتی اُس کا ذکر کسی تیسرے کے سات...,"['TWO HEARTS , 0.632']",NaN
3,1313508198876433920,.🌿.. ﴿کے میشود تمامِ جـهان سـوےِ کربـلا با ک...,"['HERB , 0.384']",NaN
4,1313510191405596928,.🌿.. ﴿کے میشود تمامِ جـهان سـوےِ کربـلا با ک...,"['HERB , 0.384']",NaN
5,1313500577578579968,.🌿.. ﴿کے میشود تمامِ جـهان سـوےِ کربـلا با ک...,"['HERB , 0.384']",NaN
6,1304899808310768128,ْ .💌 ْ ﮼اللّہُم،صلِّ،وسلمِّ،على،نبيِّنا،مُحمَ...,"['WHITE HEART SUIT , 0.669', 'LOVE LETTER , 0.5']",NaN
7,1311082338563087872,.💕کوئی گلہ کوئی شکوہ نہ رہے آپ سے یہ آرزو ہے...,"['TWO HEARTS , 0.632']",NaN
8,1313443007094545920,-- /👑 واحد اللہ کا در ہے جہاں بغیر بلاوے کے ب...,"['CROWN , 0.694']",NaN
9,1313457456723496960,'' __________🌼✨🍁 کبھی جو میرا ضبط ٹُوٹے میں ت...,"['SPARKLES , 0.351', 'BLOSSOM , 0.779', 'MAPLE...",NaN


In [ ]:
data = df['Text']
data

0               .Assalam Alikum 🦋😊 اے ایمان والو میرے دشم...
1           - '' 🍀 🌾 🌴 '' - ﺍﮬﺪِﻧَﺎﻟﺼِّﺮَﺍﻁَ ﺍَﻟﻤُﺴﺘَﻘِﯿﻢ...
2             💕 میں نہیں کرتی اُس کا ذکر کسی تیسرے کے سات...
3            .🌿.. ﴿کے میشود تمامِ جـهان سـوےِ کربـلا با ک...
4            .🌿.. ﴿کے میشود تمامِ جـهان سـوےِ کربـلا با ک...
                                 ...                        
1047995              باتیں ڈیفنس والی اوقات شاہدرے والی 😂😂😂😂
1047996    هههههههههههههههههههههههههههههههههههههههههههههه...
1047997    سیاستدان جتنی بھی کوششیں کر لیں ملک کی سلامتی ...
1047998    خان صاحب آپ صرف مہنگائی پر توجہ دیں ان اپوزیشن...
1047999     ویڈیو نا بھی ہوئی تو سکرین شاٹس لازمی ہونے ہی...
Name: Text, Length: 1048000, dtype: object

# Data Preprocessing
Removing emojis

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', str(text))

# Apply the remove_emojis function to the 'Text_with_Emojis' column
df['Text'] = df['Text'].apply(remove_emojis)

print(df['Text'])


In [ ]:
def remove_emojis(text):
  if isinstance(text, str):
    # Convert emojis to text representation
    text_with_emojis = emoji.demojize(text)
    # Remove the text that was originally representing emojis
    cleaned_text = ' '.join(word if ':' not in word else '' for word in text_with_emojis.split())
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    return cleaned_text
  else:
    # print(text)
    # cleaned_text = re.sub(r'[^\w\s]', '', text)
    return text

cleaned_tweets = [remove_emojis(tweet) for tweet in data]

In [ ]:
print(cleaned_tweets[0:5])

In [ ]:
import pandas as pd

In [ ]:
cleaned_tweets = pd.read_csv('/content/drive/MyDrive/deep/cleaned_data.csv')


In [ ]:
data = cleaned_tweets['0'].astype(str).to_list()

In [ ]:
len(data)

In [ ]:
!pip install torch
!pip install transformers[torch]
!pip install accelerate -U
!pip install --no-cache-dir transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M")
encoded_data = tokenizer(data, padding=True, truncation=True, return_tensors='pt',is_split_into_words=True,max_length=20)

# Assuming you have input and attention mask tensors
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']

# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
        }

# Create an instance of the dataset
dataset = CustomDataset(input_ids, attention_mask)

# Create a DataLoader
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "facebook/xglm-564M"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name)


ImportError: ignored

In [ ]:
# Create a DataLoader
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:



# # Load the pre-trained xglm-1.78B model and tokenizer

# # Load and preprocess your Urdu tweets dataset (replace this with your dataset loading code)
# urdu_tweets = ["Urdu tweet 1", "Urdu tweet 2", ...]  # Replace with your Urdu tweets

# # Tokenize the dataset
# encoded_urdu_tweets = tokenizer(urdu_tweets, return_tensors='pt', padding=True, truncation=True)

# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir='./xglm-fine-tuned',
    num_train_epochs=3,
    per_device_train_batch_size=8
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start fine-tuning
trainer.train()

# # Save the fine-tuned model
# model.save_pretrained('./xglm-fine-tuned')

# # Example text generation using the fine-tuned model
# input_prompt = " کے میشود تمام جـهان سـوے کربـلا با کاروان مهـدے موعد عج سفـر کنند حب_الحسین_یجمعنا"  # Replace with your desired input prompt
# input_ids = tokenizer.encode(input_prompt, return_tensors='pt')

# # Generate text using the fine-tuned model
# output = model.generate(input_ids, max_length=100, num_return_sequences=3, temperature=0.7)

# # Decode and print the generated text
# decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
# print("Generated Text:", decoded_output)

ValueError: ignored

# Model Training

In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
# Define training parameters
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)  # Adjust learning rate as needed

# Training loop
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, labels=input_ids)  # Use input_ids as labels for autoregressive training
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

TypeError: ignored

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="facebook/xglm-564M")

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

ValueError: ignored

In [ ]:
pipe( 'کے میشود تمام جـهان سـوے کربـلا با کاروان مهـدے موعد عج سفـر کنند حب_الحسین_یجمعنا')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 36, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


[{'generated_text': 'کے میشود تمام جـهان سـوے کربـلا با کاروان مهـدے موعد عج سفـر کنند حب_الحسین_یجمعنا '}]

In [ ]:
# Customize the data to be used for fine-tuning
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = TextDataset(encoded_data)

# Instantiate the Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

In [ ]:
import torch
from torch.utils.data import DataLoader

# # Example: Assuming 'train_dataset' is your training dataset
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Set up optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch['input_ids']
        targets = inputs[:, 1:]  # Shift targets by one position to predict the next token

        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


TypeError: ignored

# Start

In [ ]:
!pip install torch
!pip install transformers[torch]
!pip install sentencepiece
!pip install accelerate -U
!pip install --no-cache-dir transformers sentencepiece

In [ ]:
from transformers import Trainer, TrainingArguments,AutoTokenizer, AutoModelForCausalLM

In [ ]:
# model_name = "facebook/xglm-564M"
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
import pandas as pd

In [ ]:
cleaned_tweets = pd.read_csv('/content/drive/MyDrive/deep/cleaned_data.csv')


In [ ]:
data = cleaned_tweets['0'].astype(str).to_list()

In [ ]:
print(cleaned_tweets)

         Unnamed: 0                                                  0
0                 0  Assalam Alikum  اے ایمان والو میرے دشمنوں اور ...
1                 1         ﺍﮬﺪﻧﺎﻟﺼﺮﺍﻁ ﺍﻟﻤﺴﺘﻘﯿﻢ  ﺍﮮ ﻣﯿﺮﮮ ﺭﺏ ﻣﺠﮭﮯ ﺳﯿ...
2                 2   میں نہیں کرتی اس کا ذکر کسی تیسرے کے ساتھ اس ...
3                 3   کے میشود تمام جـهان سـوے کربـلا با کاروان مهـ...
4                 4   کے میشود تمام جـهان سـوے کربـلا با کاروان مهـ...
...             ...                                                ...
1047995     1047995                باتیں ڈیفنس والی اوقات شاہدرے والی 
1047996     1047996  هههههههههههههههههههههههههههههههههههههههههههههه...
1047997     1047997  سیاستدان جتنی بھی کوششیں کر لیں ملک کی سلامتی ...
1047998     1047998  خان صاحب آپ صرف مہنگائی پر توجہ دیں ان اپوزیشن...
1047999     1047999        ویڈیو نا بھی ہوئی تو سکرین شاٹس لازمی ہونے 

[1048000 rows x 2 columns]


In [ ]:
from torch.utils.data import DataLoader, Dataset

encoded_data = tokenizer(data, padding=True, truncation=True, return_tensors='pt',is_split_into_words=True,max_length=20)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M")
encoded_data = tokenizer(data, padding=True, truncation=True, return_tensors='pt',is_split_into_words=True,max_length=20)

# Assuming you have input and attention mask tensors
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']

# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
        }

# Create an instance of the dataset
dataset = CustomDataset(input_ids, attention_mask)

# Create a DataLoader
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(encoded_data)



{'input_ids': tensor([[   101,  77014,  13322,  10147,  13518,  36811,  12272, 107581,  10765,
            791,  13154,  11145,  29869,  47064,    771,  71604,  50744,  11363,
          23489,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
complete_sentence = batch['input_ids']
input = batch['input_ids'][:,1:-1]
target = batch['input_ids'][:,2:]
print(complete_sentence)
print(inputs)
print(target)

decoded_sentence = tokenizer.decode(complete_sentence[0])
print(f'The actual sentence \n {decoded_sentence}')
decoded_sentence = tokenizer.decode(input[0])
print(f'The inputted sentence \n {decoded_sentence}')
decoded_sentence = tokenizer.decode(target[0])
print(f'The target sentence \n {decoded_sentence}')


tensor([[   101,  77014,  13322,  10147,  13518,  36811,  12272, 107581,  10765,
            791,  13154,  11145,  29869,  47064,    771,  71604,  50744,  11363,
          23489,    102]])
tensor([[   101,  77014,  13322,  10147,  13518,  36811,  12272, 107581,  10765,
            791,  13154,  11145,  29869,  47064,    771,  71604,  50744,  11363,
          23489]])
tensor([[ 13322,  10147,  13518,  36811,  12272, 107581,  10765,    791,  13154,
          11145,  29869,  47064,    771,  71604,  50744,  11363,  23489,    102]])
The actual sentence 
 [CLS] Assalam Alikum اے ایمان والو میرے دشمنوں اور اپنے [SEP]
The inputted sentence 
 Assalam Alikum اے ایمان والو میرے دشمنوں اور اپنے
The target sentence 
 ##alam Alikum اے ایمان والو میرے دشمنوں اور اپنے [SEP]


In [ ]:
# # Example: Assuming 'train_dataset' is your training dataset
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
from tqdm import tqdm
import torch
# Set up optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 100

# Training loop
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        inputs = batch['input_ids'][:,1:-1]
        targets = batch['input_ids'][:,2:]  # Use all tokens except the last as targets  # Shift targets by one position to predict the next token

        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

